<a href="https://colab.research.google.com/github/vagabondboffin/EDAProject/blob/main/DAProject_May7th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from scipy import stats
from sklearn import preprocessing

In [3]:
data = pd.read_csv('transactionsSel.csv',engine='python')
#sort by date (not a required step since it will be automaticely done in most applications.)
data['instance_date'] = pd.to_datetime(data['instance_date'], dayfirst = True)
data["instance_date"] = data["instance_date"].astype('datetime64[ns]')
data.sort_values(by='instance_date', inplace = True)

## Adjusting prices for inflation rate

I found annual inflation rates from this link:
https://www.statista.com/statistics/297779/uae-inflation-rate/

We must indexed our data to the last year prices. I’m using the last data value to create the index, so that prices will be indexed to today’s prices. Indexed inflations are stored in 'CPI_Multiplier' column. Since our data is annual (I couldn't find monthly inflation rates/CPIs), I merged inflation data and transaction data by 'year'. Adjusted prices are in adjMeterPrice column.

The decreasing trend in inflation rates for some following years results in negative values for adjusted prices. This suggests that when considering the inflation, prices have actually dropped. 

In [4]:
inflationRate = pd.read_csv('inflationRate.csv',engine='python')
inflationRate['CPI_Multiplier'] = inflationRate['rate'].iloc[-1] / inflationRate['rate']
data = pd.merge(data, inflationRate, how='left', on='year')
data['adjMeterPrice'] = data['meter_sale_price'] * data['CPI_Multiplier']

## separate dataset by transaction types 

Previously, id and arabic columns were dropped. 

In [26]:
data['trans_group_en'].unique()
data = data.iloc[: , 1:]

In [29]:
#separate into sub-datasets
dfSales = data[data['trans_group_en'] == 'Sales']
dfGifts = data[data['trans_group_en'] == 'Gifts']
dfMortgages = data[data['trans_group_en'] == 'Mortgages']

## Sale subdataset


In [20]:
x = dfSales[['meter_sale_price','adjMeterPrice']].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_prices = pd.DataFrame(x_scaled)

In the following section, number of missing values and unique values for each attribute is reported. columns with number of unique values less than 50 are stored in selectedCol. 

In [42]:
selectedCol = []
for column in dfSales:
    numRow = dfSales.shape[0]
    print(f'attribute name: {column}')
    row = dfSales[column]
    print(f' number of missing values: {numRow - row.count()}')
    #print(f' percentage of missing values: {(numRow - row.count()) / numRow}')
    print(f' number of unique values: {row.nunique()}')
    print('+++++++++++++++++++++++++++++++++++++++++++++')
    if row.nunique() < 50:
      selectedCol.append(column)


attribute name: trans_group_en
 number of missing values: 0
 number of unique values: 1
+++++++++++++++++++++++++++++++++++++++++++++
attribute name: procedure_name_en
 number of missing values: 0
 number of unique values: 18
+++++++++++++++++++++++++++++++++++++++++++++
attribute name: property_type_en
 number of missing values: 0
 number of unique values: 4
+++++++++++++++++++++++++++++++++++++++++++++
attribute name: property_sub_type_en
 number of missing values: 137913
 number of unique values: 17
+++++++++++++++++++++++++++++++++++++++++++++
attribute name: property_usage_en
 number of missing values: 0
 number of unique values: 11
+++++++++++++++++++++++++++++++++++++++++++++
attribute name: reg_type_en
 number of missing values: 0
 number of unique values: 2
+++++++++++++++++++++++++++++++++++++++++++++
attribute name: area_name_en
 number of missing values: 0
 number of unique values: 237
+++++++++++++++++++++++++++++++++++++++++++++
attribute name: building_name_en
 number of

In [51]:
print(selectedCol)

['trans_group_en', 'procedure_name_en', 'property_type_en', 'property_sub_type_en', 'property_usage_en', 'reg_type_en', 'nearest_landmark_en', 'nearest_mall_en', 'rooms_en', 'has_parking', 'no_of_parties_role_2', 'no_of_parties_role_3', 'year', 'month']
